<img src="https://i.imgur.com/gb6B4ig.png" width="400" alt="Weights & Biases" />


This kernel is the walkthrough of the newest feature by [Weights and Biases](https://wandb.ai/site) for Dataset and Prediction Visualization called [Tables](https://docs.wandb.ai/guides/data-vis). Learn how to quickly integrate W&B in your Kaggle workflow [here](https://www.kaggle.com/ayuraj/experiment-tracking-with-weights-and-biases).

Data is at the core of every ML workflow. Tables let you visualize and query datasets and model evaluations at the example level. We can use this new tool to analyze and understand datasets, and to measure and debug model performance. I hope you will find it useful for this Kaggle competition and other competitions alike. 

This Kernel is designed with Kaggle workflow in perspective. I would love to get your suggestions around Kaggle workflow and how Tables can help with the same. 

![img](https://i.imgur.com/P7bRGmR.gif)

# 🔧 Imports and Setups

In [ ]:
# Install latest version of W&B
!pip install -q --upgrade wandb

# Import wandb
import wandb

# Login with your autorization key
wandb.login()

In [ ]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import layers
from tensorflow.keras import models
import tensorflow_addons as tfa

import os
import json
import numpy as np
import pandas as pd
pd.set_option('mode.chained_assignment', None)

from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

I will be using images of resolution 256x256 for this kernel. Thanks to [Ankur Singh](https://www.kaggle.com/ankursingh12) for this [dataset](https://www.kaggle.com/ankursingh12/resized-plant2021). Feel free to use higher resolution but that might take more time to upload.

In [ ]:
# Global variables/hyperparameters

TRAIN_PATH = '../input/resized-plant2021/img_sz_256/'
TEST_PATH = '../input/plant-pathology-2021-fgvc8/test_images/'

IMG_RES = 256
WANDB_PROJECT = 'plant-pathology-21'

In [ ]:
# Read csv file
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')

# Integer encode multi-label labels. 
labels = list(df['labels'].value_counts().keys())
labels_dict = dict(zip(labels, range(12)))
id_to_labels = {val: key for key, val in labels_dict.items()}

df = df.replace({"labels": labels_dict})
df.head()

# 1️⃣ Step 1: Upload Raw Dataset 

After understanding the problem statement of the competition, the next most important step is to have a good understanding of the dataset. A naive approach is to plot batches of data using matplotlib but its not interactive and convenient. Convenince here would be,

* to visualize as many samples as possible without running Jupyter cell(s) multiple times.
* to revisit this step multiple times without maintaining a notebook or coding again. 
* to be able to group together images by labels

📌 In the cell below the raw `train.csv` file is uploaded as [W&B Artifacts](https://wandb.ai/site/artifacts). We can even upload the entire image dataset but from a Kaggle perspective it's not very relevant since the dataset is going to be constant (more or less). 

📌 Beside uploading the csv file, I iterated over the entire dataframe and logged each image along with it's label as [W&B Tables](https://docs.wandb.ai/guides/data-vis/tables). Consider this table to be a pandas dataframe with data, meta-data and rich media all in one view. 

In [ ]:
RAW_DATA_AT = f'raw_data_{IMG_RES}'

# Initialize a new W&B run
run = wandb.init(project=WANDB_PROJECT, job_type="upload")

# create an artifact for all the raw data
raw_data_at = wandb.Artifact(RAW_DATA_AT, type="raw_data")
# Upload raw csv file as artifact
raw_data_at.add_file('../input/plant-pathology-2021-fgvc8/train.csv')

# create a table with columns we want to track/compare
preview_dt = wandb.Table(columns=["id", "image", "label"])

for i in tqdm(range(len(df))):
    image_name = df.loc[i]['image']
    label = df.loc[i]['labels']
    full_path = TRAIN_PATH+image_name
    
    # Append each example as a new row in Table.
    preview_dt.add_data(image_name, wandb.Image(full_path), id_to_labels[label])

# save artifact to W&B
raw_data_at.add(preview_dt, "data_split")
run.log_artifact(raw_data_at)
run.finish()

### [Explore the dataset interactively $\rightarrow$](https://wandb.ai/ayush-thakur/plant-pathology-21/artifacts/raw_data/raw_data_256/9e52a72a554c05224a80/files/data_split.table.json)

![img](https://i.imgur.com/TrvS9VE.png)

## 😍 Group by Labels

* Learn about the number of images per label.
* Visualize images per label interactively.
* Revisit the table again to explore more. 

![img](https://i.imgur.com/3wG0RGM.gif)

## 😍 Filter by Label

* Visualize images per label.

![img](https://i.imgur.com/iMk7j3v.gif)

# 2️⃣ Step 2: Split Raw Dataset

The most important and crucial step is to setup a reliable local validation. Weights and Biases Artifacts can be really helpful here especially with the Tables feature. 

📌 In order to build local validation, we need to split the `train.csv` file into training and validation (hold-out) split. In the cell below I have created a stratified train-val split. 

📌 This split is uploaded as an Artifact. Note the use of `use_artifact` method. It is used to establish the relationship between different artifacts. In this instance the split is created using the raw csv file that we uploaded as an artifact in the previous section.

In [ ]:
# Split into train and validation
train_df, valid_df = train_test_split(df, test_size=0.2, stratify=df['labels'].values)
print(f'Number of train images: {len(train_df)} and validation images: {len(valid_df)}')

# Add a new column
train_df.loc[:, 'split'] = 'train'
valid_df.loc[:, 'split'] = 'validation'
tmp_df = pd.concat([train_df, valid_df], axis=0, ignore_index=True)

# Save as csv files
train_df.to_csv('stratified_train.csv', index=False)
valid_df.to_csv('stratified_valid.csv', index=False)

SPLIT_DATA_AT = f'split_{IMG_RES}'

# Create new W&B run
run = wandb.init(project=WANDB_PROJECT, job_type="data_split")
# Use artifact
raw_data_at = run.use_artifact('ayush-thakur/plant-pathology-21/raw_data_256:v0', type='raw_data')

# create an artifact for all the raw data
data_split_at = wandb.Artifact(SPLIT_DATA_AT, type="stratified_split")
# Upload split as artifact
data_split_at.add_file('../input/plant-pathology-tables-support/stratified_train.csv')
data_split_at.add_file('../input/plant-pathology-tables-support/stratified_valid.csv')

# create a table with columns we want to track/compare
preview_dt = wandb.Table(columns=["id", "image", "label", "split"])

for i in tqdm(range(len(tmp_df))):
    image_name = tmp_df.loc[i]['image']
    label = tmp_df.loc[i]['labels']
    split = tmp_df.loc[i]['split']
    full_path = TRAIN_PATH+image_name
    
    # Append each example as a new row in Table.
    preview_dt.add_data(image_name, wandb.Image(full_path), id_to_labels[label], split)

# save artifact to W&B
data_split_at.add(preview_dt, "data_split")
run.log_artifact(data_split_at)
run.finish()

### [Explore Split Dataset Interactively $\rightarrow$](https://wandb.ai/ayush-thakur/plant-pathology-21/artifacts/stratified_split/split_256/a6b48c097d30d654e5ed/files/data_split.table.json)

## 😍 Group by "Split"

* Visualize images in train and validation split interactively along with the image name.
* Visualize the distribution of labels per split.

![img](https://i.imgur.com/P7bRGmR.gif)

## 😍 Group by "Labels"

* Visualize the distribution of images by split per label.

![img](https://i.imgur.com/AUBdDEr.gif)

# 3️⃣ Step 3: Train Model Using Split

With the split in place the next task is to train model(s). For sanity sake, this kernel will not go over the training proceduce. You can refer to this [kernel](https://www.kaggle.com/ayuraj/experiment-tracking-with-weights-and-biases) for a training pipeline. However, I have shown how you can use W&B Artifacts in your training pipeline to save your trained model weights. 

Note that I have trained two models (using the same split) on a GCP instance and have provided weights as [Kaggle dataset](https://www.kaggle.com/ayuraj/plant-pathology-tables-support) so that we can use it for the mentioned purpose. 

* For demonstraction purposes, we will upload only one model weight above. 
* We will use the other model for comparison later in this kernel. 

In [ ]:
# Initialize a new W&B run
run = wandb.init(project=WANDB_PROJECT, job_type="train")
# Use raw csv file
split_data_at = run.use_artifact('ayush-thakur/plant-pathology-21/split_256:v0', type='stratified_split')

# TRAIN MODEL

model_path = '../input/plant-pathology-tables-support/efficientnetb0_3.h5'
# Create an artifact to save model weights
model_at = wandb.Artifact('efficientnetb0', type="trained_model")
# Add csv file to be uploaded
model_at.add_file(model_path)
# Log the csv file
run.log_artifact(model_at)
# End W&B run
run.finish()

![img](https://i.imgur.com/mbzGEl4.png)

# 4️⃣ Step 4: Prediction Visualization

Now that we have a trained model, the obvious next step is to evaluate on the validation set. Using W&B Tables we can do just so much more than simple `model.evaluate` (Keras). I felt like I got SUPERPOWERs the first time I used this. You are up for a ride my fellow Kagglers. 

In [ ]:
# Utils
AUTOTUNE = tf.data.experimental.AUTOTUNE

CONFIG = dict (
    num_labels = 12,
    img_width = 224,
    img_height = 224,
    batch_size = 64,
)

@tf.function
def decode_image(image):
    # convert the compressed string to a 3D uint8 tensor
    image = tf.image.decode_jpeg(image, channels=3)
    # Normalize image
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    # resize the image to the desired size
    return image

@tf.function
def load_image(df_dict):
    # Load image
    image = tf.io.read_file(TRAIN_PATH+df_dict['image'])
    image = decode_image(image)
    
    # Resize image
    image = tf.image.resize(image, (CONFIG['img_height'], CONFIG['img_width']))
        
    return image

In [ ]:
# Initialize new W&B run
run = wandb.init(project=WANDB_PROJECT, job_type="prediction")

# Use split csv artifacts and download (for demonstration)
split_csv = run.use_artifact('ayush-thakur/plant-pathology-21/split_256:v0', type='stratified_split')
split_data_dir = split_csv.download()

# Use trained model and download
trained_model = artifact = run.use_artifact('ayush-thakur/plant-pathology-21/efficientnetb4:v0', type='trained_model')
model_dir = trained_model.download()

# PREPARE DATALOADER
validation_df = pd.read_csv(split_data_dir+'/stratified_valid.csv')
testloader = tf.data.Dataset.from_tensor_slices(dict(validation_df))

testloader = (
    testloader
    .map(load_image, num_parallel_calls=AUTOTUNE)
    .batch(CONFIG['batch_size'])
    .prefetch(AUTOTUNE)
)

# LOAD MODEL
tf.keras.backend.clear_session()
model = tf.keras.models.load_model(model_dir+'/efficientnetb4_0.h5')
model.summary()

# Prediction
raw_preds = []
predictions = []
for image_batch in tqdm(testloader):
    # Get model prediction
    preds = model.predict(image_batch)
    # Get label
    top1 = np.argmax(preds, axis=1)
    predictions.extend(top1)
    raw_preds.extend(preds)

# Guess
validation_df['guess'] = predictions
validation_df = validation_df.replace({"guess": id_to_labels})

# Logits
score_columns = [f'{label}_score' for label in labels_dict.keys()]
validation_df[score_columns] = raw_preds

# Initialize new artifact
prediction_at = wandb.Artifact('val_predictions_effnetb4', type="val_prediction")

columns=["id", "image", "truth", "guess"]
columns.extend(score_columns)
preview_dt = wandb.Table(columns=columns)

for i in tqdm(range(len(validation_df))):
    image_name = validation_df.loc[i]['image']
    label = validation_df.loc[i]['labels']
    guess = validation_df.loc[i]['guess']
    
    full_path = TRAIN_PATH+image_name
    
    row = [image_name, wandb.Image(full_path), id_to_labels[label], guess]
    for score in validation_df.loc[i][3:]:
        row.append(score)
        
    preview_dt.add_data(*row)

# save artifact to W&B
prediction_at.add(preview_dt, "val_prediction")
run.log_artifact(prediction_at)
run.finish()

### [Explore Prediction Validation Interactively $\rightarrow$](https://wandb.ai/ayush-thakur/plant-pathology-21/artifacts/val_prediction/val_predictions/368a98edd260ab844b24/files/val_prediction.table.json)

## 😍 Check the predictions: Group by "truth"

* The model is doing well on images with single grouth truth label. 
* `complex` label is being confused the most. Leaves (images) with too many diseases to classify visually are grouped under complex class. Need better strategy to understand what is "too many" diseases. 
* Even images with more than one label is hard for the model to clasify. Need to device better strategy to classify. 

![img](https://i.imgur.com/4pmINim.gif)

## 😍 Dive into the confusing classes: Group by "guess"

![img](https://i.imgur.com/nEJ3Use.png)

## 😍 Find images with complex class and guessed right

Let's investigate `complex` label and find out the images belonging to this label that are guesses correctly. Filter using this query: `row["truth"] = "complex" and row["guess"] = "complex"`. 

![img](https://i.imgur.com/xrFkOoD.gif)

### 😮 Group by truth - the model is not very confident in it's prediction for complex label

![img](https://i.imgur.com/aKQ8sOF.png)

# 💥 Compare Across Model Versions

Till now we visualized the model performance for one model (EfficientNetB0). With W&B Tables we can also visualize the performance across multiple models. 

Below I am comparing two models - EfficientNetB0 and EfficientNetb4. 

| Model | CV Score | LB Score | 
|:---| ----| ---: |
| EfficientNetB0 | 79.61 | 71.3 |
| EfficientNetB4 | 82.13 | 73.0 |

## 😍 Find the performance of the models grouped by ground truth

* Visualize how two models are performing on the same validation set. 
* Group by ground truth value to find the most confusing label.

![img](https://i.imgur.com/AjcfpiI.gif)

## 😍 Find score distribution across models.

* The blue color corresponds to EfficientNetB4 while yellow is EfficientNetB0.
* Clearly see the confusing labels.

![img](https://i.imgur.com/GUlVUW2.gif)

# ✨ Resources

Here are few other resources that will help you get started with Tables:

* [Visualize Data for Image Classification](https://wandb.ai/stacey/mendeleev/reports/Visualize-Data-for-Image-Classification--VmlldzozNjE3NjA)
* [Visualize Predictions over Time](https://wandb.ai/stacey/mnist-viz/reports/Visualize-Predictions-over-Time--Vmlldzo1OTQxMTk)
* [Visualize Audio Data in W&B](https://wandb.ai/stacey/cshanty/reports/Visualize-Audio-Data-in-W-B--Vmlldzo1NDMxMDk)

Check out the official documentation [here](https://docs.wandb.ai/guides/data-vis).

**Try it out yourself and let me know what more insight you found out.**